In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def scrape_hiking_project_trails(url):
    # Set up Chrome WebDriver 
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless=new")
    
    
    driver = webdriver.Chrome( options=chrome_options)
    

    # Open the webpage
    driver.get(url)
    
    def scroll_to_bottom():
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # You may adjust the sleep time based on your page loading speed
        
    while True:
        try:
            show_more_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, "load-more-trails")))
            show_more_button.click()
        except:
            scroll_to_bottom()
        else:
            scroll_to_bottom()
            
            
            # Check if the "Show More" button is still present
        is_button_present = len(driver.find_elements(By.ID, "load-more-trails")) > 0
        if not is_button_present:
            break
            


    content = driver.page_source
    driver.quit()

    soup = BeautifulSoup(content, "html.parser")
    trails = soup.find_all("tr", class_="trail-row")
    trails_info_list = []

    for trail in trails:
        trail_url = trail["data-href"]
        length_km = trail.find("span", class_="metric").text.strip().replace("\n    km", "")
        trail_title = trail.find("a", class_="text-black").text.strip()
        difficulty = trail.find("span", class_="difficulty-text text-white align-middle").text
        city = trail.find("div", class_="float-xs-right").text.strip()
        nb_reviews = trail.find("td", class_="hidden-sm-down text-muted small").text.strip()
        score_card = trail.find("span", class_="scoreStars")
        score = []

        for star_img in score_card.find_all("img"):
            star_ob = star_img["src"]
            score.append(star_ob)

        s = 5
        n = 4

        while n < len(score) and n > -1:
            if score[n] == "/img/stars/starRed.svg":
                break
            else:
                s -= 1
                n -= 1

        trails_info = {
            "name": trail_title,
            "city": city,
            "length (km)": length_km,
            "difficulty": difficulty,
            "review score": s,
            "number of reviews": nb_reviews,
            "trail url": trail_url
        }
        trails_info_list.append(trails_info)

    df_trails = pd.DataFrame(trails_info_list)
    return df_trails


In [ ]:
def scrape_data_from_multiple_pages(base_url):
    country_url = []
    
    country_page = requests.get(base_url)
    soup_country = BeautifulSoup(country_page.text, "html.parser")
    countries = soup_country.find_all("div",class_="onx-directory__item area")
    
    
    for country in countries:
        c_url = country.find("a")["href"]
        country_url.append(c_url)
        
    all_trails = []
        
    for c in country_url:
        #print(c)
        country_trail = scrape_hiking_project_trails(c)
        all_trails.append(country_trail)
        
       
        
    data = pd.concat(all_trails, ignore_index = True)
    return data

In [ ]:
all = scrape_data_from_multiple_pages("https://www.hikingproject.com")
all.to_csv("all_hiking_places.csv")